In [ ]:
# Make sure pip is up to date
!pip install --upgrade pip

# Install OpenCV
!pip3 install opencv-python-headless numpy

# Install PyTorch (CPU-only for Pi 4B)
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Install ultralytics (YOLO v8+)
!pip3 install ultralytics
